In [3]:
from random import random
import generators as gen
from math import pi, log, sin

#TIEMPOS DE ARRIBO
def lamda_t(t):
    return 30 + 30*sin((2*pi*t)/24)
    
def Poisson_NH(T):
    NT = 0
    Eventos = []
    lamda = 7
    t = -log(1 - random()) / lamda
    while t <= T:
        V = random()
        if V < lamda_t(t) / lamda:
            NT += 1
            Eventos.append(t)
        t += -log(1 - random()) / lamda
    return NT, Eventos

def exp(lamda):
    u =  random()
    return -log(u)/lamda

In [33]:
def queueSystem(hours=48):
    n, arribos =  Poisson_NH(hours)
    servicios = [exp(40) for _ in arribos]
    esperas = []
    uso = [[0,0]]
    cola = [[0,0]]

    arriboAnt = 0
    for i in range(n):
        inicio = max(arribos[i], arriboAnt)
        espera = inicio - arriboAnt
        esperas.append(espera)
        
        if espera == 0:
            uso[-1][1] += servicios[i]
            cola.append([arribos[i], cola[-1][1] + 1])
        else:
            uso.append([inicio, inicio + servicios[i]])
            cola.append([arribos[i], max(cola[-1][1] - 1, 0)])
                    
        arriboAnt = inicio + servicios[i]

    return arribos, servicios, esperas, uso, cola

queueSystem()

([0.07904957826491364,
  0.6085047289622217,
  0.6967742284852152,
  0.7737567244332365,
  1.062471918173454,
  1.3274188673439007,
  1.6175710880567378,
  1.7581319172239092,
  2.2577585234436355,
  2.4525736848420014,
  2.5200422427101743,
  2.5426346358954857,
  2.6140600316262224,
  3.1694534722433616,
  3.2158880890228168,
  3.219117238580358,
  3.3189521363272605,
  3.5965096416331424,
  3.623871039092923,
  3.6583945898994648,
  3.70474066980217,
  3.7486333997767765,
  3.959444106015419,
  3.9776309511729617,
  4.044201990060556,
  4.343048271606337,
  4.593366307873795,
  4.762823591473425,
  5.070538335733626,
  5.501826333099377,
  5.57681704818944,
  5.947350979897529,
  6.17375214175775,
  6.485269932714786,
  6.486634701490036,
  6.501128016836958,
  6.602343744287722,
  6.6095172518585334,
  6.633322721291517,
  6.688716784866546,
  6.776204503424817,
  7.137313267550094,
  7.261056484836056,
  7.304112518424192,
  7.431561345463483,
  7.495481234321467,
  7.522633584770

In [28]:
##Tasa de Utilizacion del servicio.
def tasa_servicio(usos):
    tasa = [0]*48
    tiempo_extra = 0
    for h in range(48):
        if tiempo_extra != 0:
            tasa[h] = tiempo_extra
            tiempro_extra = 0
        for uso in usos:
            if uso[0] > h:
                if uso[1] < h + 1:  #Uso en [h, h+1)
                    tasa[h] += uso[1]-uso[0]
                elif uso[0] < h + 1:  #Uso en [h, h+2), sumamos solo la parte en [h, h+1) y el extra lo guardamos para la proxima hora             
                    tiempo_extra = uso[1] - (h + 1)
                    tasa[h] += ((h + 1) - uso[0])
                else: #No hay mas usos en [h, h+1], y pasamos a la proxima hora
                    break
    return tasa

#Porcentaje de tiempo del servido ocupado
def uso_total_servicio(tasa):
    total = 0
    for t in tasa:
        total += t
    return total / 48

def promedio_tasa(Nsim):
    tasa_est = [0]*48
    for _ in range(Nsim):
        arribos, servicios, esperas, uso, cola = queueSystem()
        tasa = tasa_servicio(uso)
        for i in range(48):
            tasa_est[i] += tasa[i]
            
    for i in range(48):
        tasa_est[i] = tasa_est[i] / Nsim 
    return tasa_est

def promedio_uso(Nsim):
    r = 0
    for _ in range(Nsim):
        arribos, servicios, esperas, uso, cola = queueSystem()
        tasa = tasa_servicio(uso)
        procentaje = uso_total_servicio(tasa)

        r += procentaje
    return r / Nsim

promedio_uso(100)       
            

[0.166102824708388,
 0.18024054134772133,
 0.1773397740581391,
 0.18419082219615027,
 0.18570714242210568,
 0.1965468211436236,
 0.21312970811173165,
 0.19375222178980436,
 0.1958544697245422,
 0.20438068096530893,
 0.20537032924731413,
 0.18710598174515866,
 0.20656316134707317,
 0.19903770117189679,
 0.20809375843785965,
 0.17891105587190712,
 0.09510455437377976,
 0.04253156517082161,
 0.043957316646240766,
 0.09812649124630432,
 0.16774574355011204,
 0.2117266180841417,
 0.20281473792159094,
 0.2092152516339362,
 0.1983190384311994,
 0.1879247558198723,
 0.19166875373403877,
 0.19112737880620348,
 0.2006436179115029,
 0.20298247855314383,
 0.21257435060919236,
 0.19742578137826736,
 0.19586428945966186,
 0.1889145719776672,
 0.21047634326120035,
 0.18418140490662527,
 0.20152031478352833,
 0.19513987485583545,
 0.2075776593329214,
 0.1759955786994211,
 0.09240278760542488,
 0.03940750137058423,
 0.040452130203369734,
 0.09446282092180837,
 0.17792666487691722,
 0.18257961253327853,

In [15]:
# TIempo promedio de cliente en sistema
def est_mean(tiempos): ##EN horas es aprox 0.02499.., equivale a 90 seg.
    r = 0
    for t in tiempos:
        r += t
    return r / len(tiempos)

est_mean(servicios)

0.02476684389810057

In [ ]:
"""
QUEDA PENDIENTE:
    -Distribucion de timepos de espera
    -Histogramas de  timepo de espera
    -Dsitribucion de timepo entre arribos
    -Distribucion de servicios
--> Hay que usar la distribucion empirica para obtenerlas.
"""